In [16]:
import numpy as np
import functions as fn
from circuit_obj import Circuit

In [23]:
# Define your parameter arrays
N = 24
T = 10
circuit_realizations = 10
masks_dict = fn.load_mask_memory(N, 2)

In [25]:
def initial_state(N, n):
    """
    Generate a random state vector on N qubits with spin up at site n
    """
    up = np.array([0, 1])
    if n == 0:
        rnd_part = np.random.rand(2**(N - 1)) + 1j * np.random.rand(2**(N - 1))
        state = np.kron(up, rnd_part)
        state /= np.linalg.norm(state)
        return state
    elif n == N - 1:
        rnd_part = np.random.rand(2**(N - 1)) + 1j * np.random.rand(2**(N - 1))
        state = np.kron(rnd_part, up)
        state /= np.linalg.norm(state)
        return state
    part1 = np.random.rand(2**(n)) + 1j * np.random.rand(2**(n))
    part2 = np.random.rand(2**(N - n - 1)) + 1j * np.random.rand(2**(N - n - 1))
    state = np.kron(part1, np.kron(up, part2))
    state /= np.linalg.norm(state)
    return state

# sanity check :
# [np.round(fn.get_magnetization(initial_state(N, idx), N)[idx]) for idx in range(N)]

st = initial_state(N, 0)

In [27]:
import numpy as np
from numba import njit, prange

def make_z_masks(N):
    """
    Build an array masks[i] of length 2**N where
    masks[i][b] = +1 if the b-th basis state has spin up at site i,
                 = -1 if spin down.
    """
    dim = 1 << N  # 2**N
    masks = np.empty((N, dim), dtype=np.int8)
    for i in range(N):
        # bit i in b indicates spin-up (1) or spin-down (0)
        for b in range(dim):
            # extract the i-th bit of b:
            masks[i, b] = 1 if ((b >> (N-1-i)) & 1) else -1
    return masks

@njit(parallel=True)
def get_magnetization_numba(st_real, st_imag, masks):
    """
    Compute site magnetizations ⟨σᶻᵢ⟩ for a (possibly complex) state st.
    `st_real` and `st_imag` are the real and imaginary parts of the state vector.
    `masks` is the int8 array from make_z_masks.
    Returns a float64 array of length N.
    """
    N, dim = masks.shape
    res = np.zeros(N, np.float64)
    # compute |st|^2 once
    for i in prange(N):
        double_sum = 0.0
        for b in range(dim):
            # |st_b|^2 * mask_i[b]
            prob = st_real[b]*st_real[b] + st_imag[b]*st_imag[b]
            double_sum += prob * masks[i, b]
        res[i] = double_sum
    return res

# Wrapper to take a complex array directly:
def get_magnetization(st):
    N = int(np.log2(st.size))
    masks = make_z_masks(N)
    return get_magnetization_numba(st.real.astype(np.float64),
                                   st.imag.astype(np.float64),
                                   masks)


In [28]:
get_magnetization(st)

array([ 1.00000000e+00,  2.75785863e-06,  3.26979412e-04,  4.15794962e-04,
       -1.42453437e-04, -3.81270700e-04,  1.56931956e-05, -2.93908831e-04,
       -9.20937928e-05, -1.32802981e-04, -4.20425707e-04,  1.32342107e-04,
       -1.03810100e-04, -1.25128378e-04, -3.62836983e-04,  3.33979876e-04,
        3.17493973e-05,  3.48074691e-05, -1.76842559e-04, -1.71354927e-04,
        7.52541922e-06,  2.67299384e-04, -1.41352409e-04, -2.83775571e-05])

In [ ]:

circuits = []
circuit_realizations_max = 100

h_list_all = np.random.uniform(-np.pi, np.pi, 5*N*circuit_realizations_max).reshape(circuit_realizations_max, N, 5) 
print('Generated U1 parameters')
        

for circuit_realization in range(circuit_realizations):
    h_list = h_list_all[circuit_realization]
    gates = [fn.gen_u1([*h]) for h in h_list]
    
    assert len(gates) == N, f"Expected {N} gates, got {h_list.shape}"

    order = fn.gen_gates_order(N)    
    circuit = Circuit(N=N, gates=gates, order=order)
    circuits.append(circuit)
    
##############################################################################################

def compute_circuit(idx, circuit_real):
    circuit = circuits[circuit_real]
    state = initial_state(N, idx) # initial_state_test(theta)
    return circuit.run(masks_dict, state, T)

Generated U1 parameters


In [ ]:
compute_circuit(0, 0);

100%|██████████| 100/100 [01:18<00:00,  1.28it/s]


In [22]:
for i in range(N):
    for j in range(4):
        assert len(masks_dict[i][j]) == 2**N//4, f"mask {i} {j} has wrong shape: {len(masks_dict[i][j])} vs {2**N//4}"
print("All masks have the correct shape.")

All masks have the correct shape.
